In [1]:
import os
import random
import logging
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.transformacion import *
from utils.modelos import *

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.get_logger().setLevel(logging.ERROR)

2024-11-11 23:30:06.406404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731378606.430520   26059 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731378606.438068   26059 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 23:30:06.461520: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")

df_catalogo = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")
df_catalogo = calcular_ratio_interacciones(df_dataset, df_catalogo)
df_catalogo = calcular_puntuacion_dataset(
    dataframe=df_catalogo, 
    caracteristicas={"hito": (1, 4), "skill": (0, 15), "knowledge": (1, 15), "interaction_ratio": (0.0, 1.0)}, 
    pesos={"hito": 0.4, "skill": 0.25, "knowledge": 0.25, "interaction_ratio": 0.1}, 
    nueva_columna='score'
)

scale_complexity_norm = MinMaxScaler()
scale_complexity12_norm = MinMaxScaler()

df_catalogo['complexity_norm'] = scale_complexity_norm.fit_transform(df_catalogo[['complexity']])
df_catalogo['complexity12_norm'] = scale_complexity12_norm.fit_transform(df_catalogo[['complexity12']])

df_ratings = factorizacion_a_calificaciones(
    df_ejercicios=df_catalogo,
    matriz_factorizacion=df_dataset,
    columna_usuario='id_estudiante'
)

In [3]:
train_data, test_data = train_test_split(df_ratings, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
train_data.head(10)

,id_estudiante,id_ejercicio,nombre,h1,h2,h3,h4,s1,s2,s3,...,k4,hito,skill,knowledge,complexity,complexity12,ratio_interaccion,score,complexity_norm,complexity12_norm
1197,89,13,Cajero AutomÃ¡tico Nivel 2,0,0,0,1,0,0,0,...,1,1,1,3,19,275,0.304178,0.052381,0.090909,0.018634
8052,1099,22,Signo del ZodÃ­aco,0,0,0,1,0,0,0,...,1,1,1,1,17,273,0.848564,0.016667,0.080808,0.016563
5461,403,0,CÃ¡lculo del dÃ­gito verificador del rut,0,0,0,1,0,0,0,...,1,1,1,7,23,279,0.787206,0.123810,0.111111,0.022774
7389,1026,7,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.254569,0.000000,0.000000,0.000000
6478,923,10,Descomponer un nÃºmero,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.781984,0.035714,0.010101,0.002070
1429,104,17,Suma de los N primeros nÃºmeros naturales,0,0,0,1,0,0,0,...,0,1,0,2,2,258,0.962141,0.017857,0.005051,0.001035
4464,336,7,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.254569,0.000000,0.000000,0.000000
4092,307,4,Nota Final,0,0,0,1,0,0,0,...,1,1,0,1,1,257,0.956919,0.000000,0.000000,0.000000
2072,153,32,Cajero AutomÃ¡tico Nivel 1,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.515666,0.035714,0.010101,0.002070
8660,1159,18,Juego Adivina mi nÃºmero,0,0,0,1,0,0,0,...,1,1,0,3,3,259,0.507833,0.035714,0.010101,0.002070


In [4]:
Y_TRAIN = train_data['id_estudiante'].values
Y_TEST = test_data['id_estudiante'].values

## COMLEXITY (BINARY)
x_train_complexity_binary = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity12', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'],axis=1).values
x_test_complexity_binary = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity12', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'],axis=1).values
## COMLEXITY12 (BINARY)
x_train_complexity12_binary = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity12_binary = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values
## score (BINARY)
x_train_score_binary = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'ratio_interaccion', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_score_binary = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'ratio_interaccion', 'complexity_norm', 'complexity12_norm'], axis=1).values

## COMLEXITY (LABEL ENCODED)
x_train_complexity_label = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity_label = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values
## COMLEXITY12 (BINARY)
x_train_complexity12_label = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_complexity12_label = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values
## score (BINARY)
x_train_score_label = train_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'ratio_interaccion', 'complexity_norm', 'complexity12_norm'], axis=1).values
x_test_score_label = test_data.drop(labels=['id_estudiante', 'nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'ratio_interaccion', 'complexity_norm', 'complexity12_norm'], axis=1).values

In [5]:
epochs = 30
batch_size = 32
dropout_rate = 0.2
validation_split = 0.2
layer_units = [64, 32]
learning_rate = 0.001

# Binary models

In [6]:
model_complexity_binary = TargetModel(input_shape=x_train_complexity_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity_binary = model_complexity_binary.train(x_train_complexity_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity_binary = model_complexity_binary.evaluate(x_test_complexity_binary, Y_TEST)
predictions_complexity_binary = model_complexity_binary.predict(x_test_complexity_binary)

2024-11-11 23:30:13.687527: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 489432.6875 - val_loss: 362180.7500
Epoch 2/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 367536.1562 - val_loss: 278122.6562
Epoch 3/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 272968.2500 - val_loss: 193243.1875
Epoch 4/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 202007.5938 - val_loss: 183127.8594
Epoch 5/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 195373.8750 - val_loss: 181543.0156
Epoch 6/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 190239.5156 - val_loss: 181208.3906
Epoch 7/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 189551.2500 - val_loss: 180312.3594
Epoch 8/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 191579.3594 - val_loss: 179792.6250
Epoch 9/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 193037.9219 - val_loss: 180506.2812
Epoch 10/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 191708.5781 - val_loss: 178968.4062
Epoch 11/30
202/202 ━━━━━━━━━━━━━━━━━━━

In [7]:
model_complexity12_binary = TargetModel(input_shape=x_train_complexity12_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity12_binary = model_complexity12_binary.train(x_train_complexity12_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity12_binary = model_complexity12_binary.evaluate(x_test_complexity12_binary, Y_TEST)
predictions_complexity12_binary = model_complexity12_binary.predict(x_test_complexity12_binary)

Epoch 1/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 365447.4375 - val_loss: 256962.3125
Epoch 2/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 276957.0938 - val_loss: 255856.3594
Epoch 3/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 274013.0625 - val_loss: 253967.6719
Epoch 4/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 264888.8125 - val_loss: 251969.4219
Epoch 5/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 269084.3750 - val_loss: 251265.3594
Epoch 6/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 259583.6875 - val_loss: 246186.6406
Epoch 7/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 261035.1719 - val_loss: 240659.8281
Epoch 8/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 254681.4844 - val_loss: 232073.5156
Epoch 9/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 244564.8594 - val_loss: 220081.2969
Epoch 10/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 234634.8125 - val_loss: 205284.6094
Epoch 11/30
202/202 ━━━━━━━━━━━━━━━━━━━

In [8]:
model_score_binary = TargetModel(input_shape=x_train_score_binary.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_score_binary = model_score_binary.train(x_train_score_binary, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_score_binary = model_score_binary.evaluate(x_test_score_binary, Y_TEST)
predictions_score_binary = model_score_binary.predict(x_test_score_binary)


Epoch 1/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 494878.5625 - val_loss: 336656.5312
Epoch 2/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 281359.6875 - val_loss: 189511.8906
Epoch 3/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 196955.0625 - val_loss: 181710.4375
Epoch 4/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 194210.6406 - val_loss: 179796.6719
Epoch 5/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 190736.4219 - val_loss: 179097.3438
Epoch 6/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 187056.4219 - val_loss: 178566.9375
Epoch 7/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 186165.9688 - val_loss: 178065.1719
Epoch 8/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 187957.3750 - val_loss: 177767.6875
Epoch 9/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 187016.3594 - val_loss: 177480.0312
Epoch 10/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 188621.6250 - val_loss: 177450.3281
Epoch 11/30
202/202 ━━━━━━━━━━━━━━━━━━━

# Labels models

In [9]:
model_complexity_label = TargetModel(input_shape=x_train_complexity_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity_label = model_complexity_label.train(x_train_complexity_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity_label = model_complexity_label.evaluate(x_test_complexity_label, Y_TEST)
predictions_complexity_label = model_complexity_label.predict(x_test_complexity_label)


Epoch 1/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 490503.9062 - val_loss: 365654.6250
Epoch 2/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 374322.2188 - val_loss: 307477.9062
Epoch 3/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 301395.1562 - val_loss: 236129.3125
Epoch 4/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 241601.8281 - val_loss: 207902.4219
Epoch 5/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 218544.2812 - val_loss: 204474.2812
Epoch 6/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 218291.3125 - val_loss: 202964.4688
Epoch 7/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 212709.5312 - val_loss: 201383.9531
Epoch 8/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 207554.0469 - val_loss: 200045.0781
Epoch 9/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 209904.6875 - val_loss: 198769.3125
Epoch 10/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 215200.5781 - val_loss: 200007.2500
Epoch 11/30
202/202 ━━━━━━━━━━━━━━━━━━━

In [10]:
model_complexity12_label = TargetModel(input_shape=x_train_complexity12_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_complexity12_label = model_complexity12_label.train(x_train_complexity12_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_complexity12_label = model_complexity12_label.evaluate(x_test_complexity12_label, Y_TEST)
predictions_complexity12_label = model_complexity12_label.predict(x_test_complexity12_label)

Epoch 1/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 361909.7188 - val_loss: 257737.0625
Epoch 2/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 279754.1250 - val_loss: 257210.1094
Epoch 3/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 275220.2188 - val_loss: 257373.5469
Epoch 4/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 276797.2812 - val_loss: 256037.5156
Epoch 5/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 272679.7188 - val_loss: 256952.8438
Epoch 6/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 267557.7188 - val_loss: 255285.6562
Epoch 7/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 272089.1875 - val_loss: 254648.1562
Epoch 8/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 274641.9062 - val_loss: 253377.1875
Epoch 9/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 268278.2188 - val_loss: 252759.2188
Epoch 10/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 265969.4375 - val_loss: 253057.6094
Epoch 11/30
202/202 ━━━━━━━━━━━━━━━━━━━

In [11]:
model_score_label = TargetModel(input_shape=x_train_score_label.shape[1], learning_rate=learning_rate, dropout_rate=dropout_rate, layer_units=layer_units)
history_score_label = model_score_label.train(x_train_score_label, Y_TRAIN, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
loss_score_label = model_score_label.evaluate(x_test_score_label, Y_TEST)
predictions_score_label = model_score_label.predict(x_test_score_label)

Epoch 1/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 493537.2188 - val_loss: 343452.8438
Epoch 2/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 309798.2812 - val_loss: 247987.5156
Epoch 3/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 267732.1250 - val_loss: 231055.5312
Epoch 4/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 245684.0938 - val_loss: 214798.0312
Epoch 5/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 229252.3125 - val_loss: 202241.3125
Epoch 6/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 214956.9062 - val_loss: 195565.7344
Epoch 7/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 204917.4844 - val_loss: 192827.2188
Epoch 8/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 199486.5625 - val_loss: 191689.8125
Epoch 9/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 199772.6250 - val_loss: 191161.8125
Epoch 10/30
202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 197525.0781 - val_loss: 189864.9375
Epoch 11/30
202/202 ━━━━━━━━━━━━━━━━━━━

# Results

### Estudiante

In [12]:
# 0 | 1305
id_estudiante = 0
ejercicios_realizados = df_ratings[df_ratings['id_estudiante'] == id_estudiante]['id_ejercicio'].values
ejercicios_no_realizados = df_catalogo[~df_catalogo['id_ejercicio'].isin(ejercicios_realizados)]
df_catalogo[df_catalogo['id_ejercicio'].isin(ejercicios_realizados)].sort_values(by='complexity12', ascending=True)[['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'complexity']]

,id_ejercicio,nombre,hito,skill,knowledge,complexity
4,4,Nota Final,1,0,1,1
25,25,Ordenar tres nÃºmeros,1,0,1,1
17,17,Suma de los N primeros nÃºmeros naturales,1,0,2,2
23,23,Conversor de Decimal a Binario,1,0,2,2
3,3,NÃºmeros Primos,1,0,3,3
10,10,Descomponer un nÃºmero,1,0,3,3
18,18,Juego Adivina mi nÃºmero,1,0,3,3
26,26,Contestador AutomÃ¡tico,1,0,3,3
29,29,AprobaciÃ³n de CrÃ©ditos,1,0,3,3
42,42,NÃºmeros Amigos,1,0,3,3


### Resultados

In [13]:
predicciones_complexity_binary = model_complexity_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity12', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_complexity12_binary = model_complexity12_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_score_binary = model_score_binary.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12', 'ratio_interaccion', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()


predicciones_complexity_label = model_complexity_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity12', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_complexity12_label = model_complexity12_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'ratio_interaccion', 'score', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()
predicciones_score_label = model_score_label.predict(ejercicios_no_realizados.drop(labels=['nombre', 'id_ejercicio', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1', 'complexity', 'complexity12', 'ratio_interaccion', 'complexity_norm', 'complexity12_norm'], axis=1).values).flatten()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [14]:
recomendations_complexity_binary = ejercicios_no_realizados.copy()
recomendations_complexity_binary['predicted_preference'] = predicciones_complexity_binary
recomendations_complexity_binary = recomendations_complexity_binary.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['id_ejercicio'].isin(recomendations_complexity_binary['id_ejercicio'].values)][['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'complexity']]

,id_ejercicio,nombre,hito,skill,knowledge,complexity
5,5,Validador de Expresiones MatemÃ¡ticas,4,8,7,135
7,7,Resolver un sistema de ecuaciones,1,0,1,1
13,13,Cajero AutomÃ¡tico Nivel 2,1,1,3,19
30,30,Interpretar Expresiones MatÃ©maticas,4,12,7,199
32,32,Cajero AutomÃ¡tico Nivel 1,1,0,3,3
35,35,Factores Primos,1,1,3,19
36,36,Pintar RegiÃ³n,4,8,7,135
40,40,Juego del Gato,3,5,15,95
47,47,El problema de la mochila,4,12,7,199
51,51,Combinaciones de NÃºmeros Binarios,4,12,7,199


In [15]:
recomendations_complexity12_binary = ejercicios_no_realizados.copy()
recomendations_complexity12_binary['predicted_preference'] = predicciones_complexity12_binary
recomendations_complexity12_binary = recomendations_complexity12_binary.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['id_ejercicio'].isin(recomendations_complexity12_binary['id_ejercicio'].values)][['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'complexity12']]

,id_ejercicio,nombre,hito,skill,knowledge,complexity12
5,5,Validador de Expresiones MatemÃ¡ticas,4,8,7,1159
13,13,Cajero AutomÃ¡tico Nivel 2,1,1,3,275
20,20,MultiplicaciÃ³n de Matrices,3,4,7,839
30,30,Interpretar Expresiones MatÃ©maticas,4,12,7,1223
32,32,Cajero AutomÃ¡tico Nivel 1,1,0,3,259
35,35,Factores Primos,1,1,3,275
36,36,Pintar RegiÃ³n,4,8,7,1159
47,47,El problema de la mochila,4,12,7,1223
50,50,MCM recursivo,4,1,3,1043
51,51,Combinaciones de NÃºmeros Binarios,4,12,7,1223


In [16]:
recomendations_score_binary = ejercicios_no_realizados.copy()
recomendations_score_binary['predicted_preference'] = predicciones_score_binary
recomendations_score_binary = recomendations_score_binary.sort_values(by='predicted_preference', ascending=False).head(10)
df_catalogo[df_catalogo['id_ejercicio'].isin(recomendations_score_binary['id_ejercicio'].values)][['id_ejercicio', 'nombre', 'hito', 'skill', 'knowledge', 'score']]

,id_ejercicio,nombre,hito,skill,knowledge,score
7,7,Resolver un sistema de ecuaciones,1,0,1,0.000000
13,13,Cajero AutomÃ¡tico Nivel 2,1,1,3,0.052381
16,16,OrnitologÃ­a 1,3,0,8,0.391667
27,27,Decodificador,3,2,7,0.407143
32,32,Cajero AutomÃ¡tico Nivel 1,1,0,3,0.035714
34,34,OrnitologÃ­a 3,3,0,12,0.463095
35,35,Factores Primos,1,1,3,0.052381
43,43,Clase Usuario,3,3,15,0.566667
46,46,Pokemon,3,3,15,0.566667
52,52,OrnitologÃ­a 2,3,0,8,0.391667
